# Data QA and Cleaning

In this notebook I'm going to check if there are any evident weird values as well as basic QA.

For this case we're going to compute most of the stuff with SQL so we don't need to load the tables into the notebook and reuse the SQL files.

In [1]:
# Imports
import pandas as pd
import ftfy
from PsqlConn import create_psql_engine

In [2]:
# Useful functions
def check_field_counts(engine, base_query, table_list, column=None):
    """
    Count unique colum fields
    """
    for table in table_list:
        distinct_field_count = pd.read_sql_query(base_query.format(table=table, column = column), engine).iloc[0,0]
        print(f'For table {table} we have:\nNumber of distinct {column}: {distinct_field_count}\n')

In [3]:
# Create engine for connection
engine=create_psql_engine()

______________

## Check time interval of the sample

In [4]:
# Tables with timestamps
timestamp_dict={'content_access':'accessedat',
               'content_display':'displayedat',
               }

# Fetch timestamp query
with open('simples_table_checks/get_max_min_timestamp.sql', 'r') as file:
    timestamp_query = file.read()

# Run query for timestamp_dict tables
for table in timestamp_dict:
    print(f'Time range for {timestamp_dict[table]} for table {table}')
    display(pd.read_sql_query(timestamp_query.format(timestamp_column=timestamp_dict[table], table=table), engine).head(1))

Time range for accessedat for table content_access


,max,min
0,2020-01-31 23:59:55,2020-01-01 12:00:03


Time range for displayedat for table content_display


,max,min
0,2020-01-31 23:59:51,2020-01-01 12:00:03


________________

## Check tables size and coeherence

In [5]:
# Fetch count querys
with open('simples_table_checks/get_row_number.sql') as file:
    row_number_query=file.read()
    
with open('simples_table_checks/get_distinct_row_number.sql') as file:
    distinct_row_number_query=file.read()
    
with open('simples_table_checks/get_distinct_count_field.sql') as file:
    distinct_field_count_query=file.read()

In [6]:
# Check tables for duplicate rows or IDs
table_list = ['companies', 'content', 'content_access', 'content_display', 'industries', 'moodtracker','reasons', 'users']

for table in table_list:
    row_num = pd.read_sql_query(row_number_query.format(table=table), engine).iloc[0,0]
    distinct_row_num = pd.read_sql_query(distinct_row_number_query.format(table=table), engine).iloc[0,0]
    distinct_id_num = pd.read_sql_query(distinct_field_count_query.format(table=table, column = 'id'), engine).iloc[0,0]
    
    print(f'For table {table} we have:\nNumber of rows: {row_num}\nNumber of distinct rows: {distinct_row_num}\nNumber of distinct ids: {distinct_id_num}\n')

For table companies we have:
Number of rows: 4
Number of distinct rows: 4
Number of distinct ids: 4

For table content we have:
Number of rows: 54
Number of distinct rows: 54
Number of distinct ids: 54

For table content_access we have:
Number of rows: 200000
Number of distinct rows: 200000
Number of distinct ids: 200000

For table content_display we have:
Number of rows: 400000
Number of distinct rows: 400000
Number of distinct ids: 400000

For table industries we have:
Number of rows: 4
Number of distinct rows: 4
Number of distinct ids: 4

For table moodtracker we have:
Number of rows: 100000
Number of distinct rows: 100000
Number of distinct ids: 100000

For table reasons we have:
Number of rows: 115
Number of distinct rows: 115
Number of distinct ids: 115

For table users we have:
Number of rows: 2000
Number of distinct rows: 2000
Number of distinct ids: 2000



For all tables seen above the number of rows, distinct rows and ids is the same, suggesting that there are no duplicate rows in any table.

Let's now check if the relationship tables have a consistent amount of foreign key values with the entity tables.

In [7]:
# First we check the number of users
table_list = ['content_access', 'content_display', 'moodtracker']
check_field_counts(engine, distinct_field_count_query, table_list, 'userid')

For table content_access we have:
Number of distinct userid: 2000

For table content_display we have:
Number of distinct userid: 2000

For table moodtracker we have:
Number of distinct userid: 2000



We get 2000 distinct userid's the same value we get from the user table, so for now this seems ok. Later when we start doing joins, we'll figure out if there is a complete overlap of the user ids between these tables and the users table.

In [8]:
# Then we check the reasons
table_list = ['content', 'moodtracker']
check_field_counts(engine, distinct_field_count_query, table_list, 'reasonid')

For table content we have:
Number of distinct reasonid: 52

For table moodtracker we have:
Number of distinct reasonid: 52



There's a mismatch here, most likely caused by some reasons not being represented in the sample we have.

In [9]:
# Then we check the companies
table_list = ['users']
check_field_counts(engine, distinct_field_count_query, table_list, 'companyid')

For table users we have:
Number of distinct companyid: 4



Again, the number is consistent with the number of companies we have in the companies table.

In [10]:
# Finally we check the industry
table_list = ['users']
check_field_counts(engine, distinct_field_count_query, table_list, 'industryid')

For table users we have:
Number of distinct industryid: 4



Also consistent, since we have 4 possible different industries.

__________________

## Simple data QA checks

We already showed that the dimensions of the tables and the time intervals appear to be consistent between all tables, at least looking at them individually. We now look at the data itself.

Most of this exploration was done ad-hoc, so only examples of weird data are going to be presented here.

Out of all the tables, the content table was the only to have blatant data issues.

In [11]:
# Check #1
query1=f'''
select distinct *
from "content" c 
order by 3 desc;
'''
pd.read_sql_query(query1, engine).head(10)

,id,title,reasonid
0,83,n√£o seja sempre o culpado de tudo,"ObjectId(""5db17534d0f3fe373559ccbf"")"
1,54,qual o seu n√≠vel de intelig√™ncia emocional?,"ObjectId(""5c6c2a229c79eca733d4dd16"")"
2,72,dicas para aumentar a coragem,"ObjectId(""5c6aa1de9c79eca733d4dbea"")"
3,38,"exercite, pratique e compartilhe","ObjectId(""58335a7b514129a045865b07"")"
4,56,socialização: vocé domina essa arte?,"ObjectId(""57d9dc1ee66d040ed4e496ad"")"
5,69,o segredo da felicidade,5db17534d0f3fe373559cce1
6,45,aprenda a ser resiliente,5c6c2a2a9c79eca733d4dd23
7,27,aprenda a se conhecer,5c6c2a289c79eca733d4dd20
8,78,aprenda a se conhecer,5c6c2a279c79eca733d4dd1e
9,86,cultive a gratid√£o!,5c6c2a259c79eca733d4dd1b


As we can see above there are 3 evident problems with this table:

* The charset used for the title is not compatible with some symbols, so we get botched text. 
* The reasonid for some rows has junk text around the actual reasonid. This field is also present in the reasons and moodtracker tables, so we should double check those.
* There are duplicate values for the title field. We can confirm this by counting the number of distinct values for this field.

In [12]:
# Check duplicate title field
table_list = ['content']
check_field_counts(engine, distinct_field_count_query, table_list, 'title')

For table content we have:
Number of distinct title: 47



So we only have 47 unique titles despite having 54 distinct rows. This means that to actually compute accurate aggregate metrics we will have to fix this. Also, it is possible that there are more duplicate titles, just masked by enconding errors. We will look into this later.

In [13]:
# Check dirty reasonids
query2 = f'''
select distinct *
from moodtracker m
order by reasonid desc
limit 10;
'''
print('Moodtracker table')
display(pd.read_sql_query(query2, engine))

query3 = f'''
select distinct *
from reasons r
order by id desc;
'''
print('Reasons table')
display(pd.read_sql_query(query3, engine))

Moodtracker table


,id,userid,trackedat,mood,reasonid
0,00079eef-cae8-4d9a-983e-0452044692db,b5f4365d-2187-4b83-b217-d40c677f493a,2020-01-26 22:40:17,sad,"ObjectId(""5db17534d0f3fe373559ccbf"")"
1,002ddcd9-9081-4696-88a7-799b7251ec37,96e43d4f-fa44-49da-b2ce-09a920e8b5ab,2020-01-05 21:55:26,sad,"ObjectId(""5db17534d0f3fe373559ccbf"")"
2,00a86fa2-7af3-4f20-92ff-f6a2ecd938e6,6e6b4152-9a61-43f7-b1fb-ba230620e0ac,2020-01-23 23:46:12,happy,"ObjectId(""5db17534d0f3fe373559ccbf"")"
3,00f3ca5a-45e5-4041-88a8-5e134be29f1b,887bbeb2-874c-4ed4-bbed-90875b3243c5,2020-01-28 02:58:10,sad,"ObjectId(""5db17534d0f3fe373559ccbf"")"
4,0108a4d4-692c-49d9-9ce8-35d66e32a3c1,ad15162d-d3df-4595-a056-fe0004b2f3ec,2020-01-24 12:38:56,super-happy,"ObjectId(""5db17534d0f3fe373559ccbf"")"
5,01599c5e-87a7-4f1c-b96a-f0d88009a05c,0ce0629c-913c-410e-874c-90cdf71aa444,2020-01-11 19:03:59,sad,"ObjectId(""5db17534d0f3fe373559ccbf"")"
6,02867fe6-bcbc-47e4-9ad8-cf599bcee564,38f17d27-57ae-4bca-a292-c19bae1134bf,2020-01-14 17:10:54,ok,"ObjectId(""5db17534d0f3fe373559ccbf"")"
7,029b45c2-74c9-4529-a584-dbe75b67e829,65422eb2-dcdf-47a2-ac65-5c7040b596ea,2020-01-20 17:08:01,super-happy,"ObjectId(""5db17534d0f3fe373559ccbf"")"
8,02bc0eb1-b76c-4777-80bb-c23c1e87d129,e55c659d-6d83-41d2-8f00-1443e137bfff,2020-01-24 22:16:20,ok,"ObjectId(""5db17534d0f3fe373559ccbf"")"
9,032e575b-8036-4428-a4b7-81ff964d37c7,faf012a1-7f78-434a-8ea6-18690002aaa8,2020-01-10 18:45:34,happy,"ObjectId(""5db17534d0f3fe373559ccbf"")"


Reasons table


,id,name
0,5db17534d0f3fe373559cd02,Xenofobia
1,5db17534d0f3fe373559ccf5,Intolerância religiosa
2,5db17534d0f3fe373559cce1,Assédio moral
3,5db17534d0f3fe373559ccce,Racismo
4,5db17534d0f3fe373559ccbf,LGBTI+
...,...,...
110,57d9dc1ce66d040ed4e4969c,Anorexia
111,57d9dc1ce66d040ed4e4969b,Agressividade
112,57d9dc1ce66d040ed4e4969a,Agressão
113,57d9dc1ce66d040ed4e49699,Adoção de Filhos


So the reasonid is "dirty" in the moodtracker table as well, but it doesn't seem to be in the reasons table. Let's look for the id in `ObjectId("5db17534d0f3fe373559ccbf")` in the reasons table and in the moodtracker and contents tables.

In [14]:
query4 = f'''
select distinct *
from reasons r
where id = '5db17534d0f3fe373559ccbf'
order by id desc;
'''
print('Reasons table')
display(pd.read_sql_query(query4, engine))

Reasons table


,id,name
0,5db17534d0f3fe373559ccbf,LGBTI+


In [15]:
query5 = f'''
select distinct reasonid
from moodtracker m
where reasonid like '%%5db17534d0f3fe373559ccbf%%'
order by reasonid desc
limit 10;
'''
print('Moodtracker table')
display(pd.read_sql_query(query5, engine))

Moodtracker table


,reasonid
0,"ObjectId(""5db17534d0f3fe373559ccbf"")"


In [16]:
query5 = f'''
select distinct reasonid
from content c
where reasonid like '%%5db17534d0f3fe373559ccbf%%'
order by reasonid desc
limit 10;
'''
print('Content table')
display(pd.read_sql_query(query5, engine))

Content table


,reasonid
0,"ObjectId(""5db17534d0f3fe373559ccbf"")"


It looks like for this example the reasons table is correct, and only the moodtracker and content tables have to be fixed.

__________________
## Clean database tables

In this section we are going to clean the "dirty" data identified previously.

* We'll start by cleaning the reasonid on the moodtracker tables. 
* Then we'll clean the the reasonid in the content table.
* Finally we'll clean the title field in the content table.

### Moodtracker table

In [19]:
# This could be done with SQL, but the tables are small enough to load into memory here and I find it easier to do this with Python code.

# Get the moodtracker table
moodtracker_df = pd.read_sql_table('moodtracker', engine)
moodtracker_df.head()

,id,userid,trackedat,mood,reasonid
0,f0ed2548-dc73-45be-ac19-005331bfdd70,05733a01-7fda-4295-8ca4-1962ec5df898,2020-01-09 02:55:06,ok,57d9dc1ee66d040ed4e496a9
1,5bc270e5-2293-4eb6-b760-e2ef9e44daa2,57ccba4e-8df5-4d6e-a791-e47311196f0f,2020-01-15 15:54:53,angry,57db1d11e66d090e6410b868
2,394fb904-8c96-45e2-836e-2efdaa167c1a,5706369b-f2e5-4960-a45f-4aa22161fa3b,2020-01-22 01:39:22,angry,5c6c2a2a9c79eca733d4dd23
3,c5ede79a-f474-4a96-a6b4-0ec8119d9f73,e0be2515-b159-4dcb-9e50-85de5acefe6b,2020-01-31 12:52:46,happy,57d9dc21e66d040ed4e496c7
4,fa45550d-6a9c-4e8e-90e6-d73cc7202179,908138a6-fc5b-42cf-8cd4-0ac5ce0bdb1c,2020-01-09 19:50:06,ok,"ObjectId(""5c6c2a229c79eca733d4dd16"")"


In [28]:
# The reasonid column looks to be some kind of hash, so it should have a fixed length
display(moodtracker_df.reasonid.apply(lambda x: len(x)).value_counts())

24    91694
36     8306
Name: reasonid, dtype: int64

Around 91% of the values are 24 char wide, so we should look into the remaning ~8%

In [36]:
# Compute lengths and check which are 36 char wide
moodtracker_df["reasonid_length"] = moodtracker_df.reasonid.apply(lambda x: len(x))
faulty_reasonid = moodtracker_df.query("reasonid_length == 36").reasonid.unique()
print(f'There are {faulty_reasonid.shape[0]} ids with 36 char length')

display(faulty_reasonid)

There are 5 ids with 36 char length


array(['ObjectId("5c6c2a229c79eca733d4dd16")',
       'ObjectId("58335a7b514129a045865b07")',
       'ObjectId("5db17534d0f3fe373559ccbf")',
       'ObjectId("57d9dc1ee66d040ed4e496ad")',
       'ObjectId("5c6aa1de9c79eca733d4dbea")'], dtype=object)

All 5 reasonid suffer the same problem, they have prefix text that shouldn't be there. We are going to remove this text and replace these values with the correct ids.

In [46]:
# Since we know that our id has a fixed length, we can fix this with a very simple regex.
pattern = r'(?P<new_reasonid>[a-zA-Z0-9]{24})'
moodtracker_df["new_reasonid"] = moodtracker_df.reasonid.str.extract(pattern)

# Check how many rows are going to be updated
reasonid_toupdate = moodtracker_df.query("reasonid != new_reasonid").shape[0]
print(f"We are going to update the reasonid of {reasonid_toupdate} rows")

We are going to update the reasonid of 8306 rows


The number of rows to update is the same as the number of rows with 36 char wide reasonid, meaning that out fix covers all rows that need it.

In [47]:
# Update the table 
moodtracker_df.drop(columns = ["reasonid", "reasonid_length"], inplace=True)
moodtracker_df.rename(columns = {"new_reasonid":"reasonid"}, inplace=True)

In [61]:
# Quick check for sanity sake
print(f'''"Object" string found in the reasonid column: {moodtracker_df.reasonid.str.contains('Object').any()}''')
print(f'''There are {moodtracker_df.reasonid.apply(lambda x: len(x)).nunique()} different char lenght in the reasonid field''')
print(f'''\nLength of the reasonid values:''')
display(moodtracker_df.reasonid.apply(lambda x: len(x)).value_counts())

"Object" string found in the reasonid column: False
There are 1 different char lenght in the reasonid field

Length of the reasonid values:


24    100000
Name: reasonid, dtype: int64

Everything seems fine, so we are updating the table in the database.

In [62]:
# We are replacing the entire table in the database, since we have it all in memory
moodtracker_df.to_sql('moodtracker', engine, if_exists='replace')

### Content table

In [4]:
# Get the content table
content_df = pd.read_sql_table('content', engine)
content_df.head()

,id,title,reasonid
0,4,"dê a volta por cima, tamb√©m no trabalho",57d9dc1de66d040ed4e4969e
1,5,aumente a autoconfiança!,57d9dc1fe66d040ed4e496b3
2,6,elevando a sua autoestima,57db1d10e66d090e6410b866
3,1,como ser mais generoso,57d9dc1de66d040ed4e4969d
4,11,como lidar com a culpa?,57d9dc1ee66d040ed4e496a9


#### Reasonid
Let's do the same check we did for the moodtracker table.

In [5]:
# The reasonid column looks to be some kind of hash, so it should have a fixed length
display(content_df.reasonid.apply(lambda x: len(x)).value_counts())

24    49
36     5
Name: reasonid, dtype: int64

We find 5 "polluted" resonid, this is consistent with the 5 unique values we found previously on the moodtracker table. 

Since we are facing the same problem, we can reuse most of the code to fix the reasonid in this table.

In [8]:
# Note: We could make this a function since it is reused code, but since we are only using it twice, it's not worth the effort
pattern = r'(?P<new_reasonid>[a-zA-Z0-9]{24})'
content_df["new_reasonid"] = content_df.reasonid.str.extract(pattern)

# Check how many rows are going to be updated
reasonid_toupdate = content_df.query("reasonid != new_reasonid").shape[0]
print(f"We are going to update the reasonid of {reasonid_toupdate} rows")

# Update the table 
content_df.drop(columns = ["reasonid"], inplace=True)
content_df.rename(columns = {"new_reasonid":"reasonid"}, inplace=True)

We are going to update the reasonid of 5 rows


The number of rows we are going to update is correct, so we can update the database table.

In [9]:
# We are replacing the entire table in the database, since we have it all in memory
content_df.to_sql('content', engine, if_exists='replace')

#### Titles

Like we've seen previously, there are duplicate titles attached to different id and reasonid. Some of these are exact duplicates, while others are different strings because of encoding errors.

Because this is just a simple exercise to extract one time reports we choose to only fix the titles with botched encoding. If we were completely fixing this issue, we would have to eliminate all the duplicates, map the reasonid of the removed rows to the ones that we keep and then apply that mapping to tables that also have the reasonid.

Since this column is not a foreign key, and we only need the text to be fixed for human users, we can fix the strings and keep the different id and reasonid so the links to the other tables aren't broken. When we present results on this table we will use the title as the aggregation field, so it's ok to have duplicated titles as long as they are exactly the same.



In [12]:
# Since the table is very small we'll print the entire thing for visual inspection
content_df.sort_values("title")

,id,title,reasonid
44,73,5 passos anti-estresse,57d9dc1de66d040ed4e496a3
14,44,6 técnicas para você dominar suas emo√ß√µes,5b0d9da9abb35bed38c2ce6d
26,84,6 t√©cnicas para voc√™ dominar suas emo√ß√µes,57d9dc20e66d040ed4e496c2
20,9,afamosa uva-passa,57d9dc1ee66d040ed4e496ac
10,35,alcance a sua máxima performance,5b0d9dffabb35bed38c2ce6e
28,82,alcance a sua m√°xima performance,5c6aa1d39c79eca733d4dbd7
50,67,alivie o estresse,57d9dc1ee66d040ed4e496a8
29,81,aprenda a controlar,5c6c2a239c79eca733d4dd18
32,78,aprenda a se conhecer,5c6c2a279c79eca733d4dd1e
8,27,aprenda a se conhecer,5c6c2a289c79eca733d4dd20


In [17]:
# After visual inspections, here's the titles with duplicates and messed up encoding
index_botched_rows=[14, 26, 10, 28, 32, 8, 22, 15, 42, 16, 24, 7, 31, 45, 6, 0, 30 , 11, 33, 34]
display(content_df.iloc[index_botched_rows].sort_values("title"))

,id,title,reasonid
14,44,6 técnicas para você dominar suas emo√ß√µes,5b0d9da9abb35bed38c2ce6d
26,84,6 t√©cnicas para voc√™ dominar suas emo√ß√µes,57d9dc20e66d040ed4e496c2
10,35,alcance a sua máxima performance,5b0d9dffabb35bed38c2ce6e
28,82,alcance a sua m√°xima performance,5c6aa1d39c79eca733d4dbd7
32,78,aprenda a se conhecer,5c6c2a279c79eca733d4dd1e
8,27,aprenda a se conhecer,5c6c2a289c79eca733d4dd20
22,88,aprenda a ser resiliente,57db1d0fe66d090e6410b85e
15,45,aprenda a ser resiliente,5c6c2a2a9c79eca733d4dd23
42,77,como dormir mais r√°pido,57d9dc1ee66d040ed4e496aa
24,86,cultive a gratid√£o!,5c6c2a259c79eca733d4dd1b


In [39]:
# Check if the "clean" duplicates are actual duplicates
display(content_df.iloc[index_botched_rows].title.unique())

array(['6 técnicas para você dominar suas emo√ß√µes',
       '6 t√©cnicas para voc√™ dominar suas emo√ß√µes',
       'alcance a sua máxima performance',
       'alcance a sua m√°xima performance', 'aprenda a se conhecer',
       'aprenda a ser resiliente', 'como dormir mais r√°pido',
       'cultive a gratid√£o!', 'curta sua companhia!',
       'dicas para aumentar a coragem',
       'dê a volta por cima, tamb√©m no trabalho',
       'exercite, pratique e compartilhe', 'fa√ßa uma pausa de 2 minutos',
       'fuja do piloto autom√°tico'], dtype=object)

There are only a couple of poorly encoded titles, it's easier and faster to just replace them. Since the botched titles are like that on the source files, we would have to guess what was the original encoding of the original database/source and then fix it.

In [64]:
# We can use the ftfy package to perform a first cleaning iteration
display(content_df.iloc[index_botched_rows].title.apply(lambda x: ftfy.fix_encoding(x)))

14    6 técnicas para você dominar suas emo√ß√µes
26      6 técnicas para você dominar suas emoções
10               alcance a sua máxima performance
28              alcance a sua m√°xima performance
32                          aprenda a se conhecer
8                           aprenda a se conhecer
22                       aprenda a ser resiliente
15                       aprenda a ser resiliente
42                       como dormir mais r√°pido
16                            cultive a gratidão!
24                            cultive a gratidão!
7                            curta sua companhia!
31                           curta sua companhia!
45                  dicas para aumentar a coragem
6                   dicas para aumentar a coragem
0        dê a volta por cima, tamb√©m no trabalho
30               exercite, pratique e compartilhe
11               exercite, pratique e compartilhe
33                   fa√ßa uma pausa de 2 minutos
34                     fuja do piloto autom√°tico


This already fixes some of the poorly encoded text, so we are going to apply it to our table and manually correct the remaining since there are only 7 examples.

In [71]:
# Update dataframe
content_df["title"] = content_df["title"].apply(lambda x: ftfy.fix_encoding(x))

# Now we fix the remaining ones "manually"

title_fix_dict = {"6 técnicas para você dominar suas emo√ß√µes": '6 técnicas para você dominar suas emoções',
                 "alcance a sua m√°xima performance":'alcance a sua máxima performance',
                 "como dormir mais r√°pido": 'como dormir mais rápido',
                 "dê a volta por cima, tamb√©m no trabalho": "dê a volta por cima, também no trabalho",
                 "fa√ßa uma pausa de 2 minutos": "faça uma pausa de 2 minutos",
                 "fuja do piloto autom√°tico": "fuja do piloto automático",
                  "será que você tem depress√£o?": "será que você tem depressão?"
                 }

# Apply fix
content_df["title"].replace(title_fix_dict, inplace=True)

# Check the result
display(content_df.title.unique())

array(['dê a volta por cima, também no trabalho',
       'aumente a autoconfiança!', 'elevando a sua autoestima',
       'como ser mais generoso', 'como lidar com a culpa?',
       'virando o jogo', 'dicas para aumentar a coragem',
       'curta sua companhia!', 'aprenda a se conhecer',
       'faça a sua bússola', 'alcance a sua máxima performance',
       'exercite, pratique e compartilhe',
       'não seja sempre o culpado de tudo',
       'será que você tem depressão?',
       '6 técnicas para você dominar suas emoções',
       'aprenda a ser resiliente', 'cultive a gratidão!',
       'gerenciando o perfeccionismo',
       'qual o seu nível de inteligência emocional?',
       'você é resiliente? #seguraessamarimba', 'afamosa uva-passa',
       'pare de deixar tudo para depois',
       'passo a passo para encontrar seu propósito',
       'aprenda a controlar', 'faça uma pausa de 2 minutos',
       'fuja do piloto automático', 'se ajuda a se ajudar',
       'encontre a felicidade', '

Everything looks properly fixed now, so all that's left is to update the table in the database.

In [72]:
# We are replacing the entire table in the database, since we have it all in memory
content_df.to_sql('content', engine, if_exists='replace')